# System Testing & Debugging

Comprehensive testing suite for the Teams Clone application.

## Features:
- 🔍 Health checks and connectivity tests
- 🎯 Integration testing
- ⚡ Performance benchmarking
- 🐛 Error detection and diagnostics
- 📊 Test result reporting

## 1. Setup

In [8]:
!pip install requests pandas matplotlib seaborn time

Defaulting to user installation because normal site-packages is not writeable


ERROR: Could not find a version that satisfies the requirement time (from versions: none)

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: No matching distribution found for time


In [7]:
import requests
import time
import json
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Configuration
BASE_URL = "http://localhost:3001"
test_results = []

# Test user credentials
TEST_USER = {
    "username": f"test_user_{int(time.time())}",
    "password": "testpass123",
    "name": "Test User"
}

print("✅ Setup complete!")
print(f"Testing against: {BASE_URL}")

✅ Setup complete!
Testing against: http://localhost:3001


## 2. Helper Functions

In [ ]:
class TestResult:
    def __init__(self, name, category):
        self.name = name
        self.category = category
        self.start_time = time.time()
        self.end_time = None
        self.duration = None
        self.passed = False
        self.error = None
    
    def finish(self, passed, error=None):
        self.end_time = time.time()
        self.duration = self.end_time - self.start_time
        self.passed = passed
        self.error = error
        return self
    
    def to_dict(self):
        return {
            'name': self.name,
            'category': self.category,
            'passed': self.passed,
            'duration': round(self.duration * 1000, 2) if self.duration else 0,  # ms
            'error': str(self.error) if self.error else None
        }

def run_test(name, category, test_func):
    """Run a test and record results"""
    result = TestResult(name, category)
    try:
        test_func()
        result.finish(True)
        print(f"✅ {name} - PASSED ({result.duration*1000:.2f}ms)")
    except Exception as e:
        result.finish(False, e)
        print(f"❌ {name} - FAILED: {e}")
    
    test_results.append(result.to_dict())
    return result

def assert_status(response, expected_status):
    """Assert response status code"""
    if response.status_code != expected_status:
        raise AssertionError(f"Expected {expected_status}, got {response.status_code}: {response.text}")

def assert_json_keys(data, *keys):
    """Assert JSON response contains keys"""
    for key in keys:
        if key not in data:
            raise AssertionError(f"Missing key: {key}")

print("✅ Helper functions loaded")

## 3. Health & Connectivity Tests

In [ ]:
print("\n" + "="*60)
print("🔍 HEALTH & CONNECTIVITY TESTS")
print("="*60 + "\n")

def test_server_reachable():
    response = requests.get(BASE_URL, timeout=5)
    assert_status(response, 200)

def test_api_health():
    # Try a basic endpoint
    response = requests.get(f"{BASE_URL}/env/actions", timeout=5)
    assert response.status_code in [200, 404], "API not responding"

def test_response_time():
    start = time.time()
    requests.get(f"{BASE_URL}/env/actions", timeout=5)
    duration = time.time() - start
    assert duration < 1.0, f"Response time too slow: {duration}s"

# Run tests
run_test("Server Reachable", "Health", test_server_reachable)
run_test("API Health", "Health", test_api_health)
run_test("Response Time < 1s", "Performance", test_response_time)

## 4. Authentication Flow Tests

In [ ]:
print("\n" + "="*60)
print("🔐 AUTHENTICATION TESTS")
print("="*60 + "\n")

auth_token = None
user_id = None

def test_register():
    global user_id
    response = requests.post(
        f"{BASE_URL}/auth/register",
        json=TEST_USER
    )
    assert_status(response, 201)
    data = response.json()
    assert_json_keys(data, 'token', 'user')
    user_id = data['user'].get('id')

def test_login():
    global auth_token
    response = requests.post(
        f"{BASE_URL}/auth/login",
        json={
            "username": TEST_USER["username"],
            "password": TEST_USER["password"]
        }
    )
    assert_status(response, 200)
    data = response.json()
    assert_json_keys(data, 'token', 'user')
    auth_token = data['token']

def test_invalid_login():
    response = requests.post(
        f"{BASE_URL}/auth/login",
        json={
            "username": "invalid_user",
            "password": "wrong_password"
        }
    )
    assert response.status_code in [401, 400], "Should reject invalid credentials"

# Run tests
run_test("User Registration", "Auth", test_register)
run_test("User Login", "Auth", test_login)
run_test("Invalid Login Rejection", "Auth", test_invalid_login)

if auth_token:
    print(f"\n🔑 Auth Token: {auth_token[:20]}...")

## 5. Messages API Tests

In [ ]:
print("\n" + "="*60)
print("💬 MESSAGES API TESTS")
print("="*60 + "\n")

message_id = None

def test_send_message():
    global message_id
    response = requests.post(
        f"{BASE_URL}/api/messages",
        json={
            "channel": "general",
            "content": "Test message",
            "userId": user_id or "test-user",
            "userName": TEST_USER["name"]
        }
    )
    assert_status(response, 201)
    data = response.json()
    assert_json_keys(data, 'message')
    message_id = data['message'].get('id')

def test_get_messages():
    response = requests.get(f"{BASE_URL}/api/messages/general")
    assert_status(response, 200)
    data = response.json()
    assert isinstance(data, list), "Should return array of messages"

def test_empty_message_rejection():
    response = requests.post(
        f"{BASE_URL}/api/messages",
        json={
            "channel": "general",
            "content": "",  # Empty message
            "userId": user_id or "test-user",
            "userName": TEST_USER["name"]
        }
    )
    assert response.status_code in [400, 422], "Should reject empty messages"

# Run tests
run_test("Send Message", "Messages", test_send_message)
run_test("Get Messages", "Messages", test_get_messages)
run_test("Empty Message Rejection", "Messages", test_empty_message_rejection)

## 6. Calls API Tests

In [ ]:
print("\n" + "="*60)
print("📞 CALLS API TESTS")
print("="*60 + "\n")

call_id = None

def test_create_call():
    global call_id
    response = requests.post(
        f"{BASE_URL}/calls/create",
        json={
            "type": "video",
            "channelId": f"test-channel-{int(time.time())}",
            "userId": user_id or "test-user",
            "userName": TEST_USER["name"]
        }
    )
    assert_status(response, 201)
    data = response.json()
    assert_json_keys(data, 'call')
    call_id = data['call'].get('id')
    assert call_id is not None, "Call ID should be returned"

def test_get_call():
    if not call_id:
        raise AssertionError("No call_id available")
    response = requests.get(f"{BASE_URL}/calls/{call_id}")
    assert_status(response, 200)
    data = response.json()
    assert_json_keys(data, 'call')

def test_join_call():
    if not call_id:
        raise AssertionError("No call_id available")
    response = requests.post(
        f"{BASE_URL}/calls/{call_id}/join",
        json={
            "userId": f"user-{int(time.time())}",
            "userName": "Test Participant"
        }
    )
    assert_status(response, 200)
    data = response.json()
    assert_json_keys(data, 'call')

def test_leave_call():
    if not call_id:
        raise AssertionError("No call_id available")
    response = requests.post(
        f"{BASE_URL}/calls/{call_id}/leave",
        json={
            "userId": user_id or "test-user"
        }
    )
    assert_status(response, 200)

# Run tests
run_test("Create Call", "Calls", test_create_call)
run_test("Get Call Details", "Calls", test_get_call)
run_test("Join Call", "Calls", test_join_call)
run_test("Leave Call", "Calls", test_leave_call)

if call_id:
    print(f"\n📞 Test Call ID: {call_id}")

## 7. RL Environment Tests

In [ ]:
print("\n" + "="*60)
print("🤖 RL ENVIRONMENT TESTS")
print("="*60 + "\n")

def test_env_reset():
    response = requests.post(f"{BASE_URL}/env/reset")
    assert_status(response, 200)
    data = response.json()
    assert_json_keys(data, 'state', 'taskType')

def test_env_get_state():
    response = requests.get(f"{BASE_URL}/env/state")
    assert_status(response, 200)
    data = response.json()
    assert_json_keys(data, 'state')

def test_env_get_actions():
    response = requests.get(f"{BASE_URL}/env/actions")
    assert_status(response, 200)
    data = response.json()
    assert 'actions' in data or 'availableActions' in data, "Should return actions"

def test_env_step():
    # Reset first
    requests.post(f"{BASE_URL}/env/reset")
    
    # Take a step
    response = requests.post(
        f"{BASE_URL}/env/step",
        json={
            "action": {
                "type": "send_message",
                "channel": "general",
                "content": "Test message"
            }
        }
    )
    assert_status(response, 200)
    data = response.json()
    assert_json_keys(data, 'state', 'reward', 'done')

def test_env_stats():
    response = requests.get(f"{BASE_URL}/env/stats")
    assert_status(response, 200)
    data = response.json()
    assert 'totalEpisodes' in data or 'stats' in data, "Should return stats"

# Run tests
run_test("RL Environment Reset", "RL", test_env_reset)
run_test("Get Environment State", "RL", test_env_get_state)
run_test("Get Available Actions", "RL", test_env_get_actions)
run_test("Execute Action Step", "RL", test_env_step)
run_test("Get Environment Stats", "RL", test_env_stats)

## 8. Performance Benchmarks

In [ ]:
print("\n" + "="*60)
print("⚡ PERFORMANCE BENCHMARKS")
print("="*60 + "\n")

def benchmark_endpoint(name, method, url, data=None, iterations=10):
    """Benchmark an endpoint"""
    times = []
    
    for i in range(iterations):
        start = time.time()
        try:
            if method == 'GET':
                requests.get(url, timeout=5)
            else:
                requests.post(url, json=data, timeout=5)
            duration = (time.time() - start) * 1000  # ms
            times.append(duration)
        except Exception as e:
            print(f"  ⚠️ Request {i+1} failed: {e}")
    
    if times:
        avg = sum(times) / len(times)
        min_time = min(times)
        max_time = max(times)
        
        print(f"\n📊 {name}:")
        print(f"   Iterations: {len(times)}/{iterations}")
        print(f"   Average: {avg:.2f}ms")
        print(f"   Min: {min_time:.2f}ms")
        print(f"   Max: {max_time:.2f}ms")
        
        return times
    return []

# Benchmark different endpoints
env_actions_times = benchmark_endpoint(
    "GET /env/actions",
    "GET",
    f"{BASE_URL}/env/actions"
)

get_messages_times = benchmark_endpoint(
    "GET /api/messages/general",
    "GET",
    f"{BASE_URL}/api/messages/general"
)

# Visualize
if env_actions_times and get_messages_times:
    plt.figure(figsize=(12, 5))
    
    plt.subplot(1, 2, 1)
    plt.boxplot([env_actions_times, get_messages_times], labels=['env/actions', 'messages'])
    plt.ylabel('Response Time (ms)')
    plt.title('Response Time Distribution')
    plt.grid(True, alpha=0.3)
    
    plt.subplot(1, 2, 2)
    plt.plot(env_actions_times, marker='o', label='env/actions', alpha=0.7)
    plt.plot(get_messages_times, marker='s', label='messages', alpha=0.7)
    plt.xlabel('Request #')
    plt.ylabel('Response Time (ms)')
    plt.title('Response Time Trend')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

## 9. Integration Test - Complete User Flow

In [ ]:
print("\n" + "="*60)
print("🔄 INTEGRATION TEST - COMPLETE USER FLOW")
print("="*60 + "\n")

def test_complete_flow():
    """Test a complete user journey"""
    flow_user = {
        "username": f"flow_test_{int(time.time())}",
        "password": "testpass123",
        "name": "Flow Test User"
    }
    
    # Step 1: Register
    print("1️⃣ Registering user...")
    resp = requests.post(f"{BASE_URL}/auth/register", json=flow_user)
    assert_status(resp, 201)
    flow_user_id = resp.json()['user']['id']
    print(f"   ✅ User registered: {flow_user_id}")
    
    # Step 2: Login
    print("2️⃣ Logging in...")
    resp = requests.post(f"{BASE_URL}/auth/login", json={
        "username": flow_user["username"],
        "password": flow_user["password"]
    })
    assert_status(resp, 200)
    print("   ✅ Login successful")
    
    # Step 3: Send a message
    print("3️⃣ Sending message...")
    resp = requests.post(f"{BASE_URL}/api/messages", json={
        "channel": "general",
        "content": "Hello from integration test!",
        "userId": flow_user_id,
        "userName": flow_user["name"]
    })
    assert_status(resp, 201)
    print("   ✅ Message sent")
    
    # Step 4: Create a call
    print("4️⃣ Creating call...")
    resp = requests.post(f"{BASE_URL}/calls/create", json={
        "type": "video",
        "channelId": f"flow-test-{int(time.time())}",
        "userId": flow_user_id,
        "userName": flow_user["name"]
    })
    assert_status(resp, 201)
    flow_call_id = resp.json()['call']['id']
    print(f"   ✅ Call created: {flow_call_id}")
    
    # Step 5: Join the call
    print("5️⃣ Joining call...")
    resp = requests.post(f"{BASE_URL}/calls/{flow_call_id}/join", json={
        "userId": f"participant-{int(time.time())}",
        "userName": "Test Participant"
    })
    assert_status(resp, 200)
    print("   ✅ Joined call")
    
    # Step 6: Get call details
    print("6️⃣ Getting call details...")
    resp = requests.get(f"{BASE_URL}/calls/{flow_call_id}")
    assert_status(resp, 200)
    call_data = resp.json()['call']
    print(f"   ✅ Call has {len(call_data.get('participants', []))} participants")
    
    # Step 7: Leave the call
    print("7️⃣ Leaving call...")
    resp = requests.post(f"{BASE_URL}/calls/{flow_call_id}/leave", json={
        "userId": flow_user_id
    })
    assert_status(resp, 200)
    print("   ✅ Left call")
    
    print("\n🎉 Complete flow test PASSED!")

# Run integration test
run_test("Complete User Flow", "Integration", test_complete_flow)

## 10. Test Results Summary

In [ ]:
print("\n" + "="*60)
print("📊 TEST RESULTS SUMMARY")
print("="*60 + "\n")

if test_results:
    # Create DataFrame
    df = pd.DataFrame(test_results)
    
    # Overall stats
    total = len(df)
    passed = df['passed'].sum()
    failed = total - passed
    pass_rate = (passed / total * 100) if total > 0 else 0
    avg_duration = df['duration'].mean()
    
    print(f"Total Tests: {total}")
    print(f"Passed: {passed} ✅")
    print(f"Failed: {failed} ❌")
    print(f"Pass Rate: {pass_rate:.1f}%")
    print(f"Average Duration: {avg_duration:.2f}ms\n")
    
    # Results by category
    print("\n📋 Results by Category:")
    category_stats = df.groupby('category').agg({
        'passed': ['sum', 'count'],
        'duration': 'mean'
    }).round(2)
    category_stats.columns = ['Passed', 'Total', 'Avg Duration (ms)']
    category_stats['Pass Rate (%)'] = (category_stats['Passed'] / category_stats['Total'] * 100).round(1)
    display(category_stats)
    
    # Failed tests
    failed_tests = df[df['passed'] == False]
    if len(failed_tests) > 0:
        print("\n❌ Failed Tests:")
        for _, test in failed_tests.iterrows():
            print(f"   - {test['name']} ({test['category']})")
            print(f"     Error: {test['error']}")
    
    # Visualize results
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    
    # Pie chart - Pass/Fail
    axes[0].pie([passed, failed], labels=['Passed', 'Failed'], autopct='%1.1f%%',
                colors=['#4CAF50', '#F44336'], startangle=90)
    axes[0].set_title('Overall Test Results')
    
    # Bar chart - By category
    category_pass_rate = df.groupby('category')['passed'].apply(lambda x: (x.sum() / len(x) * 100))
    category_pass_rate.plot(kind='bar', ax=axes[1], color='skyblue')
    axes[1].set_title('Pass Rate by Category')
    axes[1].set_ylabel('Pass Rate (%)')
    axes[1].set_ylim([0, 100])
    axes[1].tick_params(axis='x', rotation=45)
    
    # Duration chart
    df_passed = df[df['passed'] == True]
    if len(df_passed) > 0:
        df_passed.plot(kind='scatter', x='name', y='duration', ax=axes[2], s=100, color='green', alpha=0.6)
        axes[2].set_title('Test Duration (Passed Tests)')
        axes[2].set_ylabel('Duration (ms)')
        axes[2].set_xlabel('')
        axes[2].tick_params(axis='x', rotation=90)
    
    plt.tight_layout()
    plt.show()
    
    # Export results
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"test_results_{timestamp}.json"
    
    report = {
        'timestamp': timestamp,
        'summary': {
            'total': total,
            'passed': passed,
            'failed': failed,
            'pass_rate': pass_rate,
            'avg_duration_ms': avg_duration
        },
        'tests': test_results
    }
    
    with open(filename, 'w') as f:
        json.dump(report, f, indent=2)
    
    print(f"\n💾 Results exported to: {filename}")
else:
    print("⚠️ No test results available")

print("\n" + "="*60)
print("✅ TESTING COMPLETE")
print("="*60)